In [1]:
import Graphics.Dynamic.Plot.R2
import Data.Complex

We want to express the following series transformation:
$$
  y(t) = \sum_{k=0}^\infty \left( \Re{x_k} \cdot \cos ((k+\tfrac12)\cdot \pi \cdot t)
                                - \Im{x_k} \cdot \sin ((k+\tfrac12)\cdot \pi \cdot t) \right).
$$

In [2]:
plotWindow $ [ plot $ continFnPlot . (.((k+1/2)*pi*))<$>[cos, negate . sin]
             | k <- [0..3] ]
          ++ [forceXRange (-0.5,0.5)]

GraphWindowSpecR2{lBound=-0.5, rBound=0.5, bBound=-1.31040238275929, tBound=1.3265344360891878, xResolution=640, yResolution=480}

In [3]:
myFourier :: [Complex Double] -> Double->Double
myFourier xs t = sum [ rx * cos ((k+1/2)*pi*t) + ix * sin ((k+1/2)*pi*t)
                     | (k, rx:+ix) <- zip [0..] xs ]

Using $(\cos,i\sin) \varphi = \tfrac12 (e^{i\cdot\varphi} \pm e^{-i\cdot\varphi})$,
$$\begin{align}
  y(t)
    =& \tfrac12 \cdot \sum_{k=0}^\infty
           \left( \Re{x_k} \cdot (e^{i\pi\cdot(k+\tfrac12)\cdot t} + e^{-i\pi\cdot(k+\tfrac12)\cdot t})
                - i\Im{x_k} \cdot (e^{i\pi\cdot(k+\tfrac12)\cdot t} - e^{-i\pi\cdot(k+\tfrac12)\cdot t}) \right)
  \\=& \tfrac12 \cdot \sum_{k=0}^\infty
           \left( (\Re{x_k} - i\Im{x_i}) \cdot e^{i\pi\cdot(k+\tfrac12)\cdot t}
                 + (\Re{x_k} + i\Im{x_i}) \cdot e^{-i\pi\cdot(k+\tfrac12)\cdot t}
                 \right)
  \\=& \sum_{k=0}^\infty
           \Re\left( x_k \cdot (e^{i\pi\cdot(k+\tfrac12)\cdot t})^\ast \right)
  \\=& \Re\left( e^{-i\frac{\pi\cdot t}2} \sum_{k=0}^\infty
           x_k \cdot e^{-i\pi\cdot k\cdot t} \right)
\end{align}$$

Discretised over points $-\tfrac12+\tfrac1{2\cdot n} , -\tfrac12 + \tfrac3{2\cdot n} \ldots \tfrac12 - \tfrac1{2\cdot n}$ (midpoints of $n$ subdivisions of the interval $[-\tfrac12,\tfrac12]$), i.e.
$$\begin{align}
  t_j = -\tfrac12 + \tfrac{1+2\cdot j}{2\cdot n}
      = \tfrac{1-n}{2\cdot n} + \tfrac{j}{n}
\end{align}$$
we get
$$\begin{align}
  y_j
    =& \Re\left( e^{-\tfrac{i\pi}2\cdot t_j} \sum_{k=0}^{n/2 - 1}
           x_k \cdot e^{-i\pi\cdot k\cdot \left(\tfrac{1-n}{2\cdot n} + \tfrac{j}{n}\right)} \right)
  \\=& \Re\left( e^{-\tfrac{i\pi}2\cdot t_j} \sum_{k=0}^{n/2 - 1}
           x_k \cdot e^{-i\pi\cdot k\cdot\tfrac{1-n}{2\cdot n}} \cdot e^{-i\pi\cdot\tfrac{k\cdot j}{n}} \right)
  \\\equiv& \Re\left( \mu_j \sum_{k=0}^{n - 1}
           \xi_k \cdot e^{-i\pi\cdot\tfrac{k\cdot j}{n}} \right)
\end{align}$$
with
$$
  \xi_k = \begin{cases}
              x_k \cdot e^{-i\pi\cdot k\cdot\tfrac{1-n}{2\cdot n}}
                 & \text{for }k<\tfrac{n}2
            \\ 0 & \text{else}
          \end{cases}
$$
and
$$
  \mu_j = e^{-\tfrac{i\pi}2\cdot t_j}.
$$
Thus, the transform can be implemented using FFT standards:

In [4]:
import Numeric.FFT.Vector.Invertible
import qualified Data.Vector.Storable as SArr

myFFourier :: [Complex Double] -> [(Double,Double)]
myFFourier xs = [ (t, realPart $ μ*υ)
                | (j,υ) <- zip [0..] $ SArr.toList (run dft ξs)
                , let μ = cis $ -pi * t/2
                      t = ((1-n)/(2*n) + j/n)]
 where ξs = SArr.fromList $
               [x * cis (-pi*k*(1-n)/(2*n)) | (k,x) <- zip [0..] xs]
             ++ (const 0<$>xs)
       n = fromIntegral $ length xs

In [5]:
someCoefs :: [Complex Double]
someCoefs = [0.4:+0.0, 0.6:+1.0, (-0.7):+(-0.6), 0.6:+(-0.9), 0.0:+1.0, 0.1:+0.1, (-0.6):+(-0.2), 0.7:+(-1.0)
            , 0, 0, 0, 0 ]

In [6]:
plotWindow [ continFnPlot $ myFourier someCoefs
           , lineSegPlot $ myFFourier someCoefs ]

GraphWindowSpecR2{lBound=-0.7777777777777778, rBound=1.7777777777777781, bBound=-5.318355746967217, tBound=5.851583402618401, xResolution=640, yResolution=480}